In [5]:
import pandas as pd
import numpy as np
import matplotlib.pyplot as plt
import seaborn as sns
import torch
from torch import nn
from torch.optim import Adam
from torch.utils.data import TensorDataset, DataLoader
import os
from tqdm import tqdm
tqdm.pandas()
from collections import Counter

In [8]:

file_path = '/Users/choedohyeon/desktop/workarea/summery/webtoon.csv'

df = pd.read_csv(file_path)
df = pd.DataFrame(df)
df = df[['title', 'outline']]
df = df.drop_duplicates(subset=['title'])
df = df.dropna()
df = df.reset_index(drop=True)

print(df)

                title                                            outline
0        회장님의 백만가지 대본  교통사고로 머리를 다친 강예성. 뇌 손상으로 자신을 소설속 남주로 착각하는 강예성을...
1           회사원 K의 비밀  직장생활 5년 차 김대리. \r\n\r\n그에겐 딱 한가지 남들에게 말하지 못하는 ...
2      황자님, 왜 잘해 주세요?  "저, 저를 어떻게 하시려는 거예요?" "강의 신에게 제물로 가는 거다." 더러운 ...
3        환상의 에덴 [개정판]  어떠한 이유로 방랑하고 있는 사형 잇테츠를 데리러 온 승려 야마토.\r\n\r\n너...
4         화려한 혼활 버스터즈  혼활에서 연전연패 중인 팔방미인 회사원, 카쿄인 요리.\r\n\r\n그리고 그녀의 ...
...               ...                                                ...
22364        060 특수부대  연재기간 : 2010.03.23~\r\n\r\n총회차 : 1047회차\r\n\r\n...
22365        0.1초의 설렘                                긴 사랑과 짧은 설렘에 대한 보고서
22366    0.0MHz  (완결)                       심령현상을 과학적으로 밝히려는 위험한 발상의 사람들
22367          -0.5˚C  아버지의 불륜으로 사랑에 거부감을 보이는 대학생 이수. 사랑을 원하여 남자들과의 만...
22368            #해모나  셀기꾼 SNS 인기스타인 모나. 어느날 그녀 앞에 랜선으로만 연애하던 남자친구가 찾...

[22369 rows x 2 columns]


/var/folders/j0/p8q34w_j4yz0ghqcplk17s240000gn/T/ipykernel_9443/478858486.py:3: DtypeWarning: Columns (6,7,10,11,13,15) have mixed types. Specify dtype option on import or set low_memory=False.
  df = pd.read_csv(file_path)


In [10]:
df['outline'] = df['outline'].replace(r"[^a-zA-Z가-힣 ]", "", regex=True)

In [47]:
df['outline'][0]

'교통사고로 머리를 다친 강예성 뇌 손상으로 자신을 소설속 남주로 착각하는 강예성을 보고 도균은 어쩔 수 없이 남편의 상황극에 맞춰준다도균은 남편과의 여러 상황극을 통해 강예성이 지금껏 겉으로 표현하지 않은 진심을 알아가게 되는데'

In [52]:
outlines = df['outline'].values
words = ' '.join(outlines).split()
words[:30]

['교통사고로',
 '머리를',
 '다친',
 '강예성',
 '뇌',
 '손상으로',
 '자신을',
 '소설속',
 '남주로',
 '착각하는',
 '강예성을',
 '보고',
 '도균은',
 '어쩔',
 '수',
 '없이',
 '남편의',
 '상황극에',
 '맞춰준다도균은',
 '남편과의',
 '여러',
 '상황극을',
 '통해',
 '강예성이',
 '지금껏',
 '겉으로',
 '표현하지',
 '않은',
 '진심을',
 '알아가게']

In [24]:
counter = Counter(words)
vocab = sorted(counter, key=counter.get, reverse=True)
int2word = dict(enumerate(vocab, 1))
int2word[0] = '<PAD>'
word2int = {word: id for id, word in int2word.items()}
word2int

{'수': 1,
 '그': 2,
 '날': 3,
 '위해': 4,
 '있는': 5,
 '된': 6,
 '한': 7,
 '없는': 8,
 '어느': 9,
 '이': 10,
 '두': 11,
 '자신의': 12,
 '같은': 13,
 '되는데': 14,
 '함께': 15,
 '이야기': 16,
 '그의': 17,
 '년': 18,
 '내': 19,
 '그리고': 20,
 '하는': 21,
 '된다': 22,
 '하지만': 23,
 '있을까': 24,
 '되고': 25,
 '다시': 26,
 '남자': 27,
 '다른': 28,
 '것을': 29,
 '모든': 30,
 '후': 31,
 '과연': 32,
 '중': 33,
 '것': 34,
 '그녀의': 35,
 '갑자기': 36,
 '내가': 37,
 '더': 38,
 '건': 39,
 '그런': 40,
 '하고': 41,
 '앞에': 42,
 '자신을': 43,
 '알게': 44,
 '가진': 45,
 '그는': 46,
 '전': 47,
 '마음을': 48,
 '않는': 49,
 '안': 50,
 '되어': 51,
 '때문에': 52,
 '점점': 53,
 '평범한': 54,
 '우연히': 55,
 '그를': 56,
 '할': 57,
 '시작된다': 58,
 '만난': 59,
 '하게': 60,
 '사람은': 61,
 '있던': 62,
 '새로운': 63,
 '걸': 64,
 '눈을': 65,
 '것은': 66,
 '바로': 67,
 '나는': 68,
 '되는': 69,
 '있다': 70,
 '온': 71,
 '그녀는': 72,
 '나타난': 73,
 '사람의': 74,
 '위한': 75,
 '못한': 76,
 '될': 77,
 '없이': 78,
 '자신이': 79,
 '채': 80,
 '로맨스': 81,
 '인해': 82,
 '사랑을': 83,
 '줄': 84,
 '대한': 85,
 '않은': 86,
 '그가': 87,
 '그녀를': 88,
 '만나게': 89,
 '게': 90,
 '왜': 91,
 '때': 9

In [26]:
outlines_enc = [[word2int[word] for word in outline.split()] for outline in tqdm(outlines)]

100%|█████████████████████████████████| 22369/22369 [00:00<00:00, 168007.40it/s]


In [46]:
sum = 0
lst = []
for i in range(22369):
    sum += len(outlines_enc[i])
    lst.append(len(outlines_enc[i]))
sum = sum / 22369
print(sum)
print(max(lst))
print(min(lst))
print(np.median(lst))

35.11618758102731
267
0
31.0


In [53]:
df['encoded'] = outlines_enc

In [54]:
outlines_enc[0]

[1277,
 2006,
 2781,
 62923,
 12545,
 26321,
 43,
 62924,
 16860,
 26322,
 62925,
 125,
 62926,
 505,
 1,
 78,
 647,
 62927,
 62928,
 3427,
 491,
 62929,
 182,
 62930,
 3428,
 6687,
 16861,
 86,
 2192,
 8372,
 14]

In [65]:
def pad_features(outlines, pad_id, seq_length=256):
    features = np.full((len(outlines), seq_length), pad_id, dtype=int)  # np.full((5, 3), 2)

    for i, row in enumerate(outlines):
        # if seq_length < len(row) then review will be trimmed
        features[i, :len(row)] = np.array(row)[:seq_length]

    return features

seq_length = 30
features = pad_features(outlines_enc, pad_id=word2int['<PAD>'], seq_length=seq_length)

assert len(features) == len(outlines_enc)
assert len(features[0]) == seq_length

In [66]:
features[0]

array([ 1277,  2006,  2781, 62923, 12545, 26321,    43, 62924, 16860,
       26322, 62925,   125, 62926,   505,     1,    78,   647, 62927,
       62928,  3427,   491, 62929,   182, 62930,  3428,  6687, 16861,
          86,  2192,  8372])

In [62]:
train_ratio = 0.7
val_ratio = 0.15
test_ratio = 0.15

# 데이터셋을 나눕니다.
train_size = int(len(df) * train_ratio)
val_size = int(len(df) * val_ratio)

train_data = df[:train_size]
val_data = df[train_size:train_size+val_size]
test_data = df[train_size+val_size:]

# 각 데이터셋의 크기를 확인합니다.
print(f"Train 데이터 수: {len(train_data)}")
print(f"Validation 데이터 수: {len(val_data)}")
print(f"Test 데이터 수: {len(test_data)}")

Train 데이터 수: 15658
Validation 데이터 수: 3355
Test 데이터 수: 3356


In [31]:
device = torch.device('cuda' if torch.cuda.is_available() else 'cpu')
print(device)

lr = 0.001
batch_size = 128
vocab_size = len(word2int)
embedding_size = 256
dropout=0.25

epochs = 8
history = {
    'train_loss': [],
    'train_acc': [],
    'val_loss': [],
    'val_acc': [],
    'epochs': epochs
}
es_limit = 5

cpu
